### <div style="text-align:center">Onboarding Historical Stats</div>

In [24]:
# importing required libraries
import pandas as pd
import mysql.connector
import pymysql
from mysql.connector import errorcode
from sqlalchemy import create_engine
import boto3
import s3fs
import datetime as dt
from datetime import datetime
import json


C:\Users\PC\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Home Value

In [25]:
#Bringing json file from Github bucket
path = r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/C__Users_elham_Desktop_New_value.json'
homeval_df = pd.read_json(path)

In [26]:
homeval_df.head()

,RegionID,RegionType,State,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,...,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31
0,61639,Zip,NY,1269494.0,1273215.0,1282550.0,1286190.0,1291949.0,1291596.0,1295069.0,...,1129365,1109779,1096961,1091641,1087154,1085121,1086397,1082450,1078831,1074640
1,84654,Zip,IL,469897.0,474329.0,476976.0,478698.0,478819.0,479880.0,482685.0,...,494846,496392,497102,497724,499152,501262,504040,506695,509141,511452
2,61637,Zip,NY,1034797.0,1038864.0,1043497.0,1047879.0,1051509.0,1051692.0,1052516.0,...,982331,989847,995933,998619,998643,1001866,1013305,1016169,1014689,1009092
3,91982,Zip,TX,339150.0,337726.0,336566.0,336298.0,336561.0,335583.0,334445.0,...,337197,337721,338625,339450,340170,341602,343191,345709,348800,353185
4,84616,Zip,IL,613823.0,618820.0,621776.0,623150.0,622324.0,622913.0,625214.0,...,635835,635617,635861,635967,637199,638751,641622,644350,647291,650118


In [27]:
#Renaming RegionID to zip code
home_value = homeval_df.rename(columns = {"RegionID":"zip_code"})

In [28]:
#Replacing nan values with zero
home_value = home_value.fillna(0)

In [29]:
#seprating the dataframe by the year
home_value1_2016 = home_value.loc[:,home_value.columns[home_value.columns.str.contains("2016")]]
home_value1_2017 = home_value.loc[:,home_value.columns[home_value.columns.str.contains("2017")]]
home_value1_2018 = home_value.loc[:,home_value.columns[home_value.columns.str.contains("2018")]]
home_value1_2019 = home_value.loc[:,home_value.columns[home_value.columns.str.contains("2019")]]
home_value1_2020 = home_value.loc[:,home_value.columns[home_value.columns.str.contains("2020")]]


In [30]:
#Finding the maximum home value in each year, adding it as new column
home_value["2016"] = home_value1_2016.mean(axis=1)
home_value["2017"] = home_value1_2017.mean(axis=1)
home_value["2018"] = home_value1_2018.mean(axis=1)
home_value["2019"] = home_value1_2019.mean(axis=1)
home_value["2020"] = home_value1_2020.mean(axis=1)


In [31]:
#Keeping column the new column(maximum value) in the dataframe
home_value = home_value.drop(['2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30',
                                       '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30',
                                       '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30',
                                       '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30',
                                       '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30',
                                       '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30',
                                       '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
                                       '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30',
                                       '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
                                       '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30',
                                       '2020-10-31', '2020-11-30', '2020-12-31'], axis = 1)

In [32]:
home_value.head()

,zip_code,RegionType,State,2016,2017,2018,2019,2020
0,61639,Zip,NY,1.289434e+06,1.248433e+06,1.246884e+06,1.251018e+06,1.104527e+06
1,84654,Zip,IL,4.818932e+05,4.949477e+05,5.023378e+05,4.966184e+05,5.001468e+05
2,61637,Zip,NY,1.047808e+06,1.041616e+06,1.034046e+06,9.618970e+05,9.990901e+05
3,91982,Zip,TX,3.349176e+05,3.324219e+05,3.358542e+05,3.371208e+05,3.416178e+05
4,84616,Zip,IL,6.254498e+05,6.408441e+05,6.486374e+05,6.415042e+05,6.393060e+05


In [33]:
df2016 = home_value[["zip_code", "2016"]].rename(columns = {'zip_code':'zipcode', '2016':'homeval'})
df2016['year'] = 2016

df2017 = home_value[["zip_code", "2017"]].rename(columns = {'zip_code':'zipcode', '2017':'homeval'})
df2017['year'] = 2017

df2018 = home_value[["zip_code", "2018"]].rename(columns = {'zip_code':'zipcode', '2018':'homeval'})
df2018['year'] = 2018

df2019 = home_value[["zip_code", "2019"]].rename(columns = {'zip_code':'zipcode', '2019':'homeval'})
df2019['year'] = 2019

df2020 = home_value[["zip_code", "2020"]].rename(columns = {'zip_code':'zipcode', '2020':'homeval'})
df2020['year'] = 2020

frames = [df2016, df2017, df2018, df2019, df2020]

homeval_df = pd.concat(frames)

homeval_df = homeval_df[['zipcode', 'year', 'homeval']]
homeval_df.head()

,zipcode,year,homeval
0,61639,2016,1.289434e+06
1,84654,2016,4.818932e+05
2,61637,2016,1.047808e+06
3,91982,2016,3.349176e+05
4,84616,2016,6.254498e+05


## Median Income

In [34]:
#Median Household Income for years 2016-2019, adding column year
Median_Income_2016 = pd.read_csv(r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/Median%20Income%202016.csv')
Median_Income_2016.insert(1, 'Year', 2016)
Median_Income_2017 = pd.read_csv(r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/Median%20Income%202017.csv')
Median_Income_2017.insert(1, 'Year', 2017)
Median_Income_2018 = pd.read_csv(r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/Median%20Income%202018.csv')
Median_Income_2018.insert(1, 'Year', 2018)
Median_Income_2019 = pd.read_csv(r'https://raw.githubusercontent.com/jaynuel/donors_dw/main/median%20income%202019.csv')
Median_Income_2019.insert(1, 'Year', 2019)

In [35]:
#Making name of dataframe's column extact same so that they can be easily concatenated
Median_Income_2016 = Median_Income_2016.rename(columns = {"Median Household income " : "Median_Household_income"})
Median_Income_2017 = Median_Income_2017.rename(columns = {"Median Household income " : "Median_Household_income"})
Median_Income_2018 = Median_Income_2018.rename(columns = {"Median Household income" : "Median_Household_income"})
Median_Income_2019 = Median_Income_2019.rename(columns = {"Median Household income" : "Median_Household_income"})

In [36]:
#Concatenating the income dataframes so that they are all in the same column
frames = [Median_Income_2016, Median_Income_2017, Median_Income_2018, Median_Income_2019]
Median_Income = pd.concat(frames)

In [37]:
#Dropping null value
Median_Income = Median_Income.dropna()

In [38]:
#Changing name of Zip Code to Zip_Code and datatype for Zip code and year to object
Median_Income = Median_Income.rename(columns = {"Zip Code": "Zip_Code"})
Median_Income.Zip_Code = Median_Income.Zip_Code.astype(object)
Median_Income.Year = Median_Income.Year.astype(object)
#Median_Income['Median_Household_income'] = Median_Income['Median_Household_income'].replace('-', '0')

In [39]:
#extract all numeric charecters from the string 'Median_Household_income' and turn it into a float64 
Median_Income['Median_Household_income'] = Median_Income['Median_Household_income'].str.extract('(\d+)', expand=False)
Median_Income['Median_Household_income'] = Median_Income['Median_Household_income'].astype(float)

In [40]:
# confirming updates
Median_Income = Median_Income.rename(columns={"Zip_Code":"zipcode", "Year":"year", "Median_Household_income":"median_income"})

Median_Income.head(2)

,zipcode,year,median_income
0,601,2016,13283.0
1,602,2016,18694.0


## Sending Dataframe to S3 Bucket (Archive)

In [41]:
# creating a variable that stores date to be append on the file to be archive
today = dt.datetime.today()
date = today.strftime("%Y%m%d")

In [42]:
#connect and load the stats data file to s3

s3pathName = 's3://yu-stats-bucket/homeval/'
s3pathName1 = 's3://yu-stats-bucket/income/'
FileNameFullPath = 'home_value' 
FileNameFullPath1 = 'income'
filenames3 = "%s%s%s.csv"%(s3pathName,FileNameFullPath,date) #name of the filepath and csv file
filenames3_1 = "%s%s%s.csv"%(s3pathName1,FileNameFullPath1,date) #name of the filepath and csv file


# Home values
homeval_df.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://yu-stats-bucket/homeval/home_value20210506.csv


In [43]:
# Median Income
Median_Income.to_csv(filenames3_1, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://yu-stats-bucket/homeval/home_value20210506.csv


## Connect to MySQL Workbench

In [44]:
# credentials for connecting to MySQL

host_name = "database-1.cvt3j4afpbrp.us-east-1.rds.amazonaws.com"
db_name = "staging_db"
u_name = "admin"
pwd = "Joli2016!"

In [45]:
# create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=db_name, user=u_name, pw=pwd))

# connect to server
connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')
print('Connected to database.')
cursor = connection.cursor()

Connected to database.


In [46]:
# call store procedure to delete data from the home value staging table
cursor.callproc('staging_db.del_homeval')
print("Home Values staging table (donation_stg) cleared.")

Home Values staging table (donation_stg) cleared.


In [47]:
# call store procedure to delete data from the donation staging table
cursor.callproc('staging_db.del_income')
print("Income staging table (donation_stg) cleared.")

Income staging table (donation_stg) cleared.


In [51]:
# loading the home value staging table (homeval_stg) in the staging database
connection.commit()
homeval_df.to_sql('homeval_stg', engine, if_exists='append', index=False)
print("Home value staging table (homeval_stg) loaded.")


Home value staging table (homeval_stg) loaded.


In [52]:
# loading the income staging table (income_stg) in the staging database
connection.commit()
Median_Income.to_sql('income_stg', engine, if_exists='append', index=False)
print("Income staging table (income_stg) loaded.")


Income staging table (income_stg) loaded.


### Loading Donation Table in the Donation Database

In [53]:
# connect to stats_db
donation_db = "stats_db"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=donation_db, user=u_name, pw=pwd))

connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')

print('Connected to database.')
cursor = connection.cursor()

Connected to database.


In [54]:
# call stored procedures to load dimenstions in stats_db

cursor.callproc('stats_db.load_dim_homeval')
print("Home value dimension table (dim_homeval) in stats database (stats_db) loaded.")

cursor.callproc('stats_db.load_dim_income')
print("Income dimension table (dim_income) in stats database (stats_db) loaded.")

cursor.callproc('stats_db.load_dim_location')
print("Location dimension table (dim_location) in stats database (stats_db) loaded.")

cursor.callproc('stats_db.load_stats_fact')
print("Stats Fact table (stats_fact) in stats database (stats_db) loaded.")


Home value dimension table (dim_homeval) in stats database (stats_db) loaded.
Income dimension table (dim_income) in stats database (stats_db) loaded.
Location dimension table (dim_location) in stats database (stats_db) loaded.
Stats Fact table (stats_fact) in stats database (stats_db) loaded.


In [55]:
# commiting and closing database connection
connection.commit()
connection.close()
print('Disconnected from database.')

Disconnected from database.
